In [1]:
!kubecolor -n elasticsearch get all

NAME                               READY   STATUS    RESTARTS   AGE
pod/elasticsearch-coordinating-0   1/1     Running   0          4h13m
pod/elasticsearch-coordinating-1   1/1     Running   0          4h13m
pod/elasticsearch-data-0           1/1     Running   0          4h13m
pod/elasticsearch-data-1           1/1     Running   0          4h13m
pod/elasticsearch-ingest-0         1/1     Running   0          4h13m
pod/elasticsearch-ingest-1         1/1     Running   0          4h13m
pod/elasticsearch-master-0         1/1     Running   0          4h13m
pod/elasticsearch-master-1         1/1     Running   0          4h13m

NAME                                    TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)             AGE
service/elasticsearch                   ClusterIP   10.105.31.246   <none>        9200/TCP,9300/TCP   4h13m
service/elasticsearch-coordinating-hl   ClusterIP   None            <none>        9200/TCP,9300/TCP   4h13m
service/elasticsearch-data-hl           ClusterIP

In [2]:
!kubectl -n elasticsearch  get service elasticsearch -o yaml | yq

apiVersion: v1
kind: Service
metadata:
  annotations:
    meta.helm.sh/release-name: elasticsearch
    meta.helm.sh/release-namespace: elasticsearch
  creationTimestamp: "2024-02-02T10:41:46Z"
  labels:
    app.kubernetes.io/component: coordinating-only
    app.kubernetes.io/instance: elasticsearch
    app.kubernetes.io/managed-by: Helm
    app.kubernetes.io/name: elasticsearch
    app.kubernetes.io/version: 8.12.0
    helm.sh/chart: elasticsearch-19.17.2
  name: elasticsearch
  namespace: elasticsearch
  resourceVersion: "26535"
  uid: a3ae2930-a1d2-40ae-9b80-517193072fd4
spec:
  clusterIP: 10.105.31.246
  clusterIPs:
    - 10.105.31.246
  internalTrafficPolicy: Cluster
  ipFamilies:
    - IPv4
  ipFamilyPolicy: SingleStack
  ports:
    - name: tcp-rest-api
      port: 9200
      protocol: TCP
      targetPort: rest-api
    - name: tcp-transport
      port: 9300
      protocol: TCP
      targetPort: 9300
  selector:
    app.kubernetes.io/component: coordinating-only
    app.kubernetes

In [4]:
!nmap -p 9200,9300 elasticsearch.elasticsearch.svc

Starting Nmap 7.80 ( https://nmap.org ) at 2024-02-02 12:02 -03
Nmap scan report for elasticsearch.elasticsearch.svc (10.105.31.246)
Host is up (0.00012s latency).
rDNS record for 10.105.31.246: elasticsearch.elasticsearch.svc.cluster2.xpt

PORT     STATE SERVICE
9200/tcp open  wap-wsp
9300/tcp open  vrace

Nmap done: 1 IP address (1 host up) scanned in 0.11 seconds


   * https://www.elastic.co/guide/en/elasticsearch/reference/current/rest-apis.html: Elasticsearch exposes REST APIs that are used by the UI components and can be called directly to configure and access Elasticsearch features.
      * https://www.elastic.co/guide/en/elasticsearch/reference/current/security-api.html 

In [11]:
!curl -s -u username:password -X GET "http://elasticsearch.elasticsearch.svc:9200/elastic/user?pretty" | jq

{
  "error": "Incorrect HTTP method for uri [/_elastic/user?pretty] and method [GET], allowed: [POST]",
  "status": 405
}


In [31]:
!curl -s -u elastic:password -X GET "http://elasticsearch.elasticsearch.svc:9200/_cluster/health?pretty" | jq

{
  "cluster_name": "elastic",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 8,
  "number_of_data_nodes": 2,
  "active_primary_shards": 37,
  "active_shards": 74,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100
}


In [14]:
!curl -s -u username:password -X GET "http://elasticsearch.elasticsearch.svc:9200/_security/user?pretty" | jq

{
  "error": "Incorrect HTTP method for uri [/_security/user?pretty] and method [GET], allowed: [POST]",
  "status": 405
}


In [24]:
!curl -s -u elastic: -X GET "http://elasticsearch.elasticsearch.svc:9200/_security/admin" | jq


{
  "error": "Incorrect HTTP method for uri [/_security/admin] and method [GET], allowed: [POST]",
  "status": 405
}


In [22]:
!curl -s -u username:password -X PUT "http://elasticsearch.elasticsearch.svc:9200/_security/user/elastic"  -H "Content-Type: application/json" \
 -d '{ \
  "password" : "user_password", \
  "roles" : [ "role1", "role2" ], \
  "full_name" : "Elastic User", \
  "email" : "elastic@example.com", \
  "metadata" : { \
    "intelligence" : 7 \
  } \
}'

{"error":"no handler found for uri [/_security/user/elastic] and method [PUT]"}